In [5]:
import pandas as pd
import numpy as np
from tqdm import tqdm

In [7]:
one_run = pd.read_csv('/home/MarkF/DivideConquer/Results/10_Runs/'
                 'ica_run_1_independent_components.tsv', sep='\t', index_col=0)
one_run.columns = [f'{x}_1' for x in one_run.columns]
ten_runs = pd.read_csv('/home/MarkF/DivideConquer/Results/10_Runs/'
                 'ica_independent_components_consensus.tsv', sep='\t', index_col=0)
ten_runs.columns = [f'{x}_10' for x in ten_runs.columns]


eleven_runs = pd.read_csv('/home/MarkF/DivideConquer/Results/11_Runs/'
                 'ica_independent_components_consensus.tsv', sep='\t', index_col=0)
eleven_runs.columns = [f'{x}_11' for x in eleven_runs.columns]
twenty_five_runs = pd.read_csv('/home/MarkF/DivideConquer/Results/MathExperiment/2_Split/One_Normalized/ICARUN_ALL/'
                 'ica_independent_components_consensus.tsv', sep='\t', index_col=0)
twenty_five_runs.columns = [f'{x}_25' for x in twenty_five_runs.columns]
fifty_runs = pd.read_csv('/home/MarkF/DivideConquer/Results/50_Runs/'
                 'ica_independent_components_consensus.tsv', sep='\t', index_col=0)
fifty_runs.columns = [f'{x}_50' for x in fifty_runs.columns]
hunderd_runs = pd.read_csv('/home/MarkF/DivideConquer/Results/100_Runs/'
                 'ica_independent_components_consensus.tsv', sep='\t', index_col=0)
hunderd_runs.columns = [f'{x}_100' for x in hunderd_runs.columns]


In [8]:
one_run.shape

(19635, 1996)

In [9]:
ten_runs.shape

(19635, 1912)

In [10]:
eleven_runs.shape

(19635, 165)

In [11]:
twenty_five_runs.shape

(19635, 171)

In [12]:
fifty_runs.shape

(19635, 179)

In [13]:
hunderd_runs.shape

(19635, 170)

In [14]:
df = twenty_five_runs.join(fifty_runs)
df = df.join(hunderd_runs)
df = df.join(eleven_runs)
df = df.join(ten_runs)
df = df.join(one_run)
correlation = np.corrcoef(df.values, rowvar=False)
correlation = np.absolute(correlation)
correlation = pd.DataFrame(correlation, columns=df.columns, index=df.columns)
#correlation = correlation.loc[twenty_five_runs.columns, fifty_runs.columns]

In [15]:
import seaborn as sns
import matplotlib.pyplot as plt

In [25]:
cols = hunderd_runs.columns
#index = [x for x in correlation.columns if x not in cols]
index =  one_run.columns
sum(correlation.loc[index, cols].max(axis=1).sort_values() < 0.6)

1844

In [26]:
cols = hunderd_runs.columns
#index = [x for x in correlation.columns if x not in cols]
index =  ten_runs.columns
sum(correlation.loc[index, cols].max(axis=1).sort_values() < 0.6)

1747

In [27]:
cols = hunderd_runs.columns
#index = [x for x in correlation.columns if x not in cols]
index =  eleven_runs.columns
sum(correlation.loc[index, cols].max(axis=1).sort_values() < 0.6)

5

In [28]:
cols = hunderd_runs.columns
#index = [x for x in correlation.columns if x not in cols]
index =  twenty_five_runs.columns
sum(correlation.loc[index, cols].max(axis=1).sort_values() < 0.6)

7

In [29]:
cols = hunderd_runs.columns
#index = [x for x in correlation.columns if x not in cols]
index =  fifty_runs.columns
sum(correlation.loc[index, cols].max(axis=1).sort_values() < 0.6)

10

In [13]:
cols = hunderd_runs.columns
#index = [x for x in correlation.columns if x not in cols]
index =  fifty_runs.columns
sum(correlation.loc[index, cols].max(axis=0).sort_values() < 0.6)

1

In [19]:
index =  eleven_runs.columns
correlation.loc[index, cols].max(axis=0).sort_values()

consensus independent component 152_100    0.026596
consensus independent component 169_100    0.027262
consensus independent component 156_100    0.048159
consensus independent component 164_100    0.065778
consensus independent component 163_100    0.081092
                                             ...   
consensus independent component 55_100     0.969339
consensus independent component 67_100     0.969464
consensus independent component 50_100     0.971058
consensus independent component 59_100     0.972404
consensus independent component 54_100     0.983553
Length: 170, dtype: float64

In [73]:
from bokeh.models.sources import ColumnDataSource
from bokeh.plotting import figure
from bokeh.io import output_notebook, show, push_notebook
output_notebook()

Loading BokehJS ...

In [ ]:
old_df = pd.read_csv(f'/home/MarkF/DivideConquer/Results/100_Runs/'
                 f'ica_run_1_independent_components.tsv', sep='\t', index_col=0)
old_df.columns = [f'{x}_1' for x in old_df.columns]
end_df = old_df.copy()

# Figure
my_figure = figure(plot_width=800, plot_height=400)
test_data = ColumnDataSource(data=dict(x=[1], y=[old_df.shape[1]]))
line = my_figure.line("x", "y", source=test_data)
handle = show(my_figure, notebook_handle=True)


x = []
y = []


for i in range(2,101):
    df_new = pd.read_csv(f'/home/MarkF/DivideConquer/Results/100_Runs/'
                 f'ica_run_{i}_independent_components.tsv', sep='\t', index_col=0)
    df_new.columns = [f'{x}_{i}' for x in df_new.columns]
    end_df = end_df.join(df_new)
    correlation = np.corrcoef(end_df.values, rowvar=False)
    correlation = np.absolute(correlation)
    correlation = pd.DataFrame(correlation, columns=end_df.columns, index=end_df.columns)
    index = [x for x in end_df.columns if f'_{i}' in x]
    columns = [x for x in end_df.columns if x not in index]
    correlation = correlation.loc[index, columns]
    x.append(i)
    y.append(((correlation.max(axis=1)) <.9).sum())
    # Figure
    #new_data['x'] = x  
    #new_data['y'] = y 
    test_data.stream({ "x": [x[-1]], "y": [y[-1]] })
    push_notebook(handle=handle)